In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
mass_of_carbon = 12.0107  # amu

filename = '/Users/Howl/Documents/Alcohol/carbon_lattice/carbon_lattice.xyz'

xyz = open(filename, 'r')
atoms = []
coords = []
box_dimensions = [14.84, 9.96, 9.00]

n_atoms = int(xyz.readline())
title = xyz.readline()
for line in xyz:
    atom,x,y,z = line.split()
    atoms.append(atom)
    coords.append([float(x), float(y), float(z)])
#print(coords)

#def update_pos(coords, vel, acc, dt, box_length):
    """
    Update the particle positions accounting for the 
    periodic boundary condition.
    
    Parameters
    ----------
    coords: ndarray of floats
        The positions of the particles in 3 dimensions
    vel: ndarray of floats
        The velocities of the particles in 3 dimensions
    acc: ndarray of floats
        The accelerations of the particles in 3 dimensions
    dt: float
        The timestep length
    box_dimensions: float 
        The size of the unit cell
    
    Returns
    -------
    ndarray of floats:
        New positions of the particles in a single dimension
    """

vel = 0.01
acc = 0.001
dt = 0.1
x = []
y = []
z = []
    
for i in range(len(coords)):
    new_x = coords[i][0] + vel * dt + 0.5 * acc * dt * dt
    new_y = coords[i][1] + vel * dt + 0.5 * acc * dt * dt
    new_z = coords[i][2] + vel * dt + 0.5 * acc * dt * dt
    x.append(new_x)
    y.append(new_y)
    z.append(new_z)

newx = []
newy = []
newz = []

for i in range(len(x)):
    new_posx = x[i] % box_dimensions[0]
    newx.append(new_posx)
for i in range(len(y)):
    new_posy = y[i] % box_dimensions[1]
    newy.append(new_posy)
for i in range(len(z)):
    new_posz = z[i] % box_dimensions[2]
    newz.append(new_posz)
    
new_pos = np.dstack([newx, newy, newz])
    #return new_pos
print(new_pos)

[[[1.005000e-03 1.005000e-03 1.005000e-03]
  [1.005000e-03 1.005000e-03 4.501005e+00]
  [1.005000e-03 4.981005e+00 1.005000e-03]
  [1.005000e-03 4.981005e+00 4.501005e+00]
  [7.421005e+00 1.005000e-03 1.005000e-03]
  [7.421005e+00 1.005000e-03 4.501005e+00]
  [7.421005e+00 4.981005e+00 1.005000e-03]
  [7.421005e+00 4.981005e+00 4.501005e+00]
  [3.711005e+00 1.005000e-03 2.251005e+00]
  [3.711005e+00 4.981005e+00 2.251005e+00]]]


In [ ]:

def lj_force(r, epsilon, sigma):
    """
    Implementation of the Lennard-Jones potential 
    to calculate the force of the interaction.
    
    Parameters
    ----------
    r: float
        Distance between two particles (Å)
    epsilon: float 
        Potential energy at the equilibrium bond 
        length (eV)
    sigma: float 
        Distance at which the potential energy is 
        zero (Å)
    
    Returns
    -------
    float
        Force of the van der Waals interaction (eV/Å)
    """
    return 48 * epsilon * np.power(
        sigma / r, 13) - 24 * epsilon * np.power(
        sigma / r, 7)

def get_accelerations(positions, box_length, cutoff):
    """
    Calculate the acceleration on each particle as a 
    result of each other particle. 
    
    Parameters
    ----------
    positions: ndarray of floats
        The positions, in a single dimension, for all
        of the particles
    box_length: float 
        The size of the periodic cell
    cutoff: float
        The distance after which the interaction 
        is ignored
        
    Returns
    -------
    ndarray of floats
        The acceleration on each particle
    """
    accel_x = np.zeros((positions.size, positions.size))
    for i in range(0, positions.size - 1):
        for j in range(i + 1, positions.size):
            r_x = positions[j] - positions[i]
            r_x = r_x % box_length
            rmag = np.sqrt(r_x * r_x)
            force_scalar = lj_force(rmag, 0.0103, 3.4)
            force_x = force_scalar * r_x / rmag
            accel_x[i, j] = force_x / mass_of_argon
            accel_x[j, i] = - force_x / mass_of_argon
    return np.sum(accel_x, axis=0)